In [1]:
import pandas as pd
import numpy as np
import joblib as jb

import tensorflow
import keras
import matplotlib.pyplot as plt
from keras import callbacks
from keras.models import Model
from keras.layers import Dense, Input, Conv2D, MaxPool2D, UpSampling2D

from keras.engine.topology import Layer, InputSpec
from keras import backend as k
from keras import regularizers

Using TensorFlow backend.


In [2]:
# load preprocessed dragonfly images
dragonpath = r'D:\Linnaeus_models\dragon\train\dragon_train.npy'
dragon_train = np.load(dragonpath)
print(f'Number of images: {len(dragon_train)}')

Number of images: 13807


##  Only

In [5]:
# separate into training set and validation set and reshape to fit the NN
x_train = dragon_train[0:12500] #.reshape(-1, 65536)
x_val = dragon_train[12500:] #.reshape(-1, 65536)

In [6]:
x_train = np.expand_dims(x_train, axis=-1)
x_val = np.expand_dims(x_val, axis=-1)

In [7]:
# this is our input placeholder
input_img = Input(shape=(256, 256, 1))

# "encoded" is the encoded representation of the input
encoded = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
encoded = MaxPool2D((2, 2), padding='same')(encoded)
encoded = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
encoded = MaxPool2D((2, 2), padding='same')(encoded)
encoded = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
encoded = MaxPool2D((2, 2), padding='same')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
decoded = UpSampling2D((2, 2))(decoded)
decoded = Conv2D(16, (3, 3), activation='relu', padding='same')(decoded)
decoded = UpSampling2D((2, 2))(decoded)
decoded = Conv2D(32, (3, 3), activation='relu', padding='same')(decoded)
decoded = UpSampling2D((2, 2))(decoded)
decoded = Conv2D(1, (3, 3), padding='same')(decoded)


# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

In [ ]:
autoencoder.summary()

In [ ]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

autoencoder.compile(optimizer='Adam', loss='mse')

train_history = autoencoder.fit(x_train, x_train, epochs=1, batch_size=150, 
                                validation_data=(x_val, x_val))




Train on 12500 samples, validate on 1307 samples
Epoch 1/1
10350/12500 [=======================>......] - ETA: 4:17 - loss: 0.0296